# Upload clips to Zooniverse
Script to generate short clips from underwater movies and upload them to Zooniverse.

## Initial requirements
Import required packages, indicate how many clips to generate, the folder to store the new clips and generate the "manuscript" (a csv file needed to upload the clips to Zooniverse).

In [ ]:
# Import required packages
import os
import subprocess
import math
import csv
from datetime import date
from panoptes_client import Panoptes, Project #needed to upload clips to Zooniverse

# Specify how many clips to generate and its length (seconds)
n_clips = 1000
clip_length = 10

# Specify the folder location to store new clips
if not os.path.exists("clips"):
    os.makedirs("clips")

# Create an empty csvfile to store the metadata required to upload clips to Zooniverse
manuscript_name = "manuscript_" + date.today().strftime("%d_%m_%Y") + ".csv"

with open('clips/' + manuscript_name, 'wb') as subject_metadata:
    filewriter = csv.writer(subject_metadata, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['clip_id','filename', '#start_time', '#end_time'])

## Create a list of the clips to generate

In [ ]:
# Query the koster lab database to randomly select n clips from movie sections that haven't been clipped yet
clip_list


## Generate the clips

In [ ]:
# Generate one clip at the time and update the koster lab database and manuscript
for clip in clip_list:
    # Generate and store the clip
    fileoutput = "clips/" + str(clip_filename) + "_" + str(int(clip_start)) + ".mp4"
    subprocess.call(["ffmpeg", "-ss", str(clip_start), "-t", clip_duration, "-i", clip_movie, "-c", "copy", "-force_key_frames", "1", fileoutput])
    
     # Add clip information to the koster lab database
    clip_id = 
    filename = clip_filename
    start_time = clip_start
    end_time = clip_start + clip_duration
    clip_date = date.today().strftime("%d_%m_%Y")
    
    # Add information to the manuscript to keep track of the clips generated
    file_output = fileoutput.split("/")[-1]
    row = [clip_id, file_output, split_start, end_time]
    with open('clips/' + manuscript_name,'a') as subject_metadata:
        writer = csv.writer(subject_metadata)
        writer.writerow(row)
        subject_metadata.close()   

## Upload clips to Zooniverse
we will use the [Python SDK for Panoptes](https://github.com/zooniverse/panoptes-python-client) to upload the clips we have just generated. 
Note, only Zooniverse project admins can upload clips to the Koster lab Zooniverse project. 

In [ ]:
# Connect to Zooniverse with your username and password
Panoptes.connect(username='', password='')

# Specify the project number of the koster lab
koster_project = Project(9747)

# Create a subject set (the Zooniverse data set that will store the clips)
subject_set = SubjectSet()

subject_set.links.project = koster_project
subject_set.display_name = 'clips_'+ date.today().strftime("%d_%m_%Y")

subject_set.save()

# Upload the clips to the project
new_subjects = []

with open('clips/' + manuscript_name,'a') as subject_metadata:
    reader = csv.reader(subject_metadata, skipinitialspace=True)
    for filename, metadata in subject_metadata.items():
        subject = Subject()

        subject.links.project = koster_project
        subject.add_location(filename)

        subject.metadata.update(metadata)

        subject.save()
        new_subjects.append(subject)